# INITIAL SETUP WORK

In [ ]:
!pip3 install virtualenv
!virtualenv rag_llm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 36.5 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 738ms
  creator CPython3Posix(dest=/content/rag_llm, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.1, setuptools==70.1.0, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


We activate the virtual environment

In [ ]:
!source /content/rag_llm/bin/activate;

## Installing the Libraries

In this part of the code we install the necessary libraries to run the code

In [27]:

%pip install --upgrade --quiet session-info langchain pipreqs openai  tiktoken  python-dotenv transformers    langchain-community langchain-openai neo4j
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import session_info
session_info.show()

In [ ]:
#!pip freeze > requirements.txt

In [ ]:
!pipreqs .

INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt

We will use OPENAI embeddings for this section to generate the embeddidings

We connect to the NEO4j database. We are using the cloud version in this instance but you could easily use a local version and run it using docker. We go to the neo4J site and use the database in the sandbox so we will leave the credentials open.

You select the option

In [28]:
url ="bolt://44.200.56.144:7687"
username ="neo4j"
password = "displacement-rug-vector"
database = "neo4j"


If you want to upload a smaller version of the data feel free to use the code below that allows you to download the data and use a smaller version of the dataset and create your own version. The small dataset does not have as many relationships as the large dataset in the default NEO4J database which allows us to calculate things such as Revenue and Ratings

#### How you would create nodes and relationships in the data

In [1]:
import pandas as pd

# URL of the CSV file
csv_url = 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_url)

# Display the first few rows of the DataFrame
print(df.head())


   movieId    released                        title  \
0        1  1995-11-22                    Toy Story   
1        2  1995-12-15                      Jumanji   
2        3  1995-12-22             Grumpier Old Men   
3        4  1995-12-22            Waiting to Exhale   
4        5  1995-12-08  Father of the Bride Part II   

                                              actors         director  \
0         Jim Varney|Tim Allen|Tom Hanks|Don Rickles    John Lasseter   
1  Robin Williams|Bradley Pierce|Kirsten Dunst|Jo...     Joe Johnston   
2  Walter Matthau|Ann-Margret|Jack Lemmon|Sophia ...    Howard Deutch   
3  Whitney Houston|Lela Rochon|Angela Bassett|Lor...  Forest Whitaker   
4  Steve Martin|Kimberly Williams-Paisley|Diane K...    Charles Shyer   

                                        genres  imdbRating  
0  Adventure|Animation|Children|Comedy|Fantasy         8.3  
1                   Adventure|Children|Fantasy         6.9  
2                               Comedy|Romance 


# Import movie information

"""<br>
This code iterates through each tow and creates the following nodes: Director, Movie, Actor, Person and Genre

The nodes with properties are Movie which has the properties; released, titel and imdbRating

THE relationships defined in the code are DIRECTED, ACTED_IN, IN_GENRE

<br>
<br>
"""
movies_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row <br>
MERGE (m:Movie {id:row.movieId}) <br>
SET m.released = date(row.released), <br>
    m.title = row.title,  <br>
    m.imdbRating = toFloat(row.imdbRating) <br>
FOREACH (director in split(row.director, '|') |  <br>
    MERGE (p:Person {name:trim(director)}) <br>
    MERGE (p)-[:DIRECTED]->(m)) <br>
FOREACH (actor in split(row.actors, '|') |  <br>
    MERGE (p:Person {name:trim(actor)}) <br>
    MERGE (p)-[:ACTED_IN]->(m)) <br>
FOREACH (genre in split(row.genres, '|') |  <br>
    MERGE (g:Genre {name:trim(genre)}) <br>
    MERGE (m)-[:IN_GENRE]->(g)) <br>
""" <br>
 <br>
graph.query(movies_query)

In [29]:
##we load the the movie data into NEO4j


from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url = url,
    username = username,
    password = password,
    database = database

)


In [13]:
movies_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

## Graph Schema

Here we look at the graph schema and it shows the expansive relationships, nodes and node properties available in the dataset

In [32]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {imdbRating: FLOAT, embedding: LIST, id: STRING, released: DATE, title: STRING}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


We validate if the model is working correctly
|

## We create the Vector Embeddings

Vector embeddings will help us query our data better

In [34]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"

In [35]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] =  userdata.get('OPEN_API_KEY')

In [17]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model=embeddings_model),
    url=url,
    username=username,
    password=password,
    index_name='movies',
    node_label="Movie",
    text_node_properties=['title', 'released', 'imdbRating'],
    embedding_node_property='embedding',
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [36]:
response = vector_index.similarity_search(
    "what rating does the movie Casino have"
)

In [37]:
response

[Document(page_content='\ntitle: Casino\nreleased: 1995-11-22\nimdbRating: 8.2'),
 Document(page_content='\ntitle: Showgirls\nreleased: 1995-09-22\nimdbRating: 4.6'),
 Document(page_content='\ntitle: Leaving Las Vegas\nreleased: 1996-02-09\nimdbRating: 7.6'),
 Document(page_content='\ntitle: Fair Game\nreleased: 1995-11-03\nimdbRating: 4.1')]

In [22]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.invoke(
    "what rating does the movie Casino have?"
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'The movie "Casino" has an IMDb rating of 8.2.'

In [38]:
vector_qa.invoke(
    "what is the highest rated movie?"
)


{'query': 'what is the highest rated movie?',
 'result': 'The highest rated movie among the provided list is "Seven" with an IMDb rating of 8.6.'}

# Using Cypher Search

In [23]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    qa_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), graph=graph, verbose=True
)


In [39]:
cypher_chain.invoke(
    "Who acted in the movie Casino"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Casino"})
RETURN p.name
Full Context:
[{'p.name': 'Sharon Stone'}, {'p.name': 'Joe Pesci'}, {'p.name': 'James Woods'}, {'p.name': 'Robert De Niro'}]

> Finished chain.


{'query': 'Who acted in the movie Casino',
 'result': 'Sharon Stone, Joe Pesci, James Woods, Robert De Niro acted in the movie Casino.'}

## Knowledge Agent

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# Using with chat history


system = '''Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
Introduce yourself first and say Hi, I am your local movie boffin and I am here to answer all your questions about movies. Please proceed to ask me a question
'''



human = '''{input}

{agent_scratchpad}

(reminder to respond in a JSON blob no matter what)'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
    ]
)

In [51]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import AgentExecutor, create_structured_chat_agent

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.run,
        description="""Useful when you need to answer questions about movies
        and their title as well as their rating and the year they were released
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions movies and their genres, their directors
        or any complex caluclations that you would need to such as counting movies people appear in.
        Use full question as input.
        """,
    )
]

In [52]:
from langchain_core.messages import AIMessage, HumanMessage
agent = create_structured_chat_agent(ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


agent_executor.invoke({"input": "Who acts in the movie Casino",
        })


{'input': 'Who acts in the movie Casino',
 'output': 'Robert De Niro, Sharon Stone, and Joe Pesci star in the movie Casino.'}

In [40]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.run,
        description="""Useful when you need to answer questions about movies
        and their title as well as their rating and the year they were released
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions movies and their genres, their directors
        or any complex caluclations that you would need to such as counting movies people appear in.
        Use full question as input.
        """,
    )
]

mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
response = mrkl.run("Which team is assigned to maintain PaymentService?")
print(response)

In [ ]:
##we load the the movie data into NEO4j


from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url = url,
    username = username,
    password = password,
    database = database

)

movies_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)



##defining vector similarity and vector databses

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = "text-embedding-3-small"


import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] =  userdata.get('OPEN_API_KEY')


vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model=embeddings_model),
    url=url,
    username=username,
    password=password,
    index_name='movies',
    node_label="Movie",
    text_node_properties=['title', 'released', 'imdbRating'],
    embedding_node_property='embedding',
)



from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.invoke(
    "what rating does the movie Casino have?"
)



from langchain.chains import GraphCypherQAChain

##defining cypherchain
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    qa_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), graph=graph, verbose=True
)

##Defining agents

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.run,
        description="""Useful when you need to answer questions about movies
        and their title as well as their rating and the year they were released
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions movies and their genres, their directors
        or any complex caluclations that you would need to such as counting movies people appear in.
        Use full question as input.
        """,
    )
]

mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)




In [26]:
response = mrkl.run("Which team is assigned to maintain PaymentService?")
print(response)

NameError: name 'mrkl' is not defined